In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import random
from datetime import date
try:
    from urlparse import urljoin  # Python2
except ImportError:
    from urllib.parse import urljoin  # Python3

In [2]:
# Baixando o sitemap.xml do site
print('baixando sitemap.xml...')
url="https://www.foxterciaimobiliaria.com.br/sitemap.xml"
response = requests.get(url)
with open('sitemap.xml', 'wb') as file:
    file.write(response.content)
print('sitemap.xml ok!')

baixando sitemap.xml...
sitemap.xml ok!


In [3]:

def union(tocrawl,newlinks):
    for e in newlinks:
        if e not in tocrawl:
            tocrawl.append(e)
    return   

                  
# Encontra todos os links na conteúdo Soup e guarda em alllinks
def get_all_links(page):
    links = []
    for link in page_content(page).find_all('a'):
        links.append(str(link.get('href')))
    return links
                  
                  
def check_links(alllinks):
    links_checked = []

    for link_to_check in alllinks:

        if link_to_check[:38] == url_seed:
            links_checked.append(link_to_check)
        
        if link_to_check[:8] == url_segment:
            link_to_check_join = urljoin(url_seed, link_to_check)
            links_checked.append(link_to_check_join)
            
    return links_checked    

# Coletando conteúdo de uma página
def page_content(page):

    response = ''
    while response == '':
        wait = 0
        try:
            response = requests.get(page)
        except:
            wait = random.randint(10,2000) # caso o servidor não responda
            print("Connection refused by the server..")
            print("Let me sleep for " + str(wait) + " seconds")
            print("ZZzzzz...")
            time.sleep(wait)
            print("Was a nice sleep, now let me continue...")
            continue
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    
    # ignorar se página sem id de imóvel
    if soup.find('span', {'itemprop':"identifier"}) is None:
        return soup
    
    #coletar id
    id_imovel = soup.find('span', {'itemprop':"identifier"})
    id_clean = id_imovel.get_text()[7:]
    id_list.append(id_clean)
        
        
    #coletar preço
    try:        
        price = soup.find('span', {'itemprop': "price"})
        price_clean0 = price.get_text()[3:]
        price_clean = price_clean0.replace(".", "")
    except:
        price_clean = 'NA'
    price_list.append(price_clean)
    
        
    #coletar área privativa
    try:
        area = soup.find(string='Área privativa').next_element.next_element
        area_clean = area.get_text()[:-2]
    except:
        area_clean = 'NA'        
    area_list.append(area_clean)
        
        
    #coletar bairro
    try:
        district = soup.find(string='Bairro').next_element.next_element
        district_clean = district.get_text()
    except:
        district_clean = 'NA'
    district_list.append(district_clean)
        
        
    #coletar cidade
    try:
        city = soup.find(string='Cidade').next_element.next_element
        city_clean = city.get_text()
    except:
        city_clean = 'NA'
    city_list.append(city_clean)

        
    #coletar tipo
    try:
        type1 = soup.find('span', {'itemprop': "category"})
        type1_clean = type1.get_text()
    except:
        type1_clean = 'NA'
    type1_list.append(type1_clean)


    #coletar segmento
    try:
        segment = soup.find(string='Segmento').next_element.next_element
        segment_clean = segment.get_text()
    except:
        segment_clean = 'NA'
    segment_list.append(segment_clean)

        
    #coletar valor do condomínio
    try:
        condominium = soup.find(string='Condomínio').next_element.next_element
        condominium_clean0 = condominium.get_text()[3:]
        condominium_clean = condominium_clean0.replace(".", "")
    except:
        condominium_clean = 'NA'
    condominium_list.append(condominium_clean)

        
    #coletar IPTU
    try:
        iptu = soup.find(string='IPTU Anual').next_element.next_element
        iptu_clean0 = iptu.get_text()[3:]
        iptu_clean = iptu_clean0.replace(".", "")
    except:
        iptu_clean = 'NA'
    iptu_list.append(iptu_clean)
    
        
    #coletar dormitórios
    try:
        rooms = soup.find(string='Dormitórios').next_element.next_element
        rooms_clean = rooms.get_text()
    except:
        rooms_clean = 'NA'
    rooms_list.append(rooms_clean)

        
    #coletar vagas para estacionamento
    try:
        box = soup.find('span', {'class': "value vagas"})
        box_clean = box.get_text()
    except:
        box_clean = 'NA'
    box_list.append(box_clean)
                  
        
    #coletar url
    url_list.append(page)
    
    #salvando data
    date_list.append(date_today)
    
    return soup


In [4]:
# Iniciando visita e captura
def crawl_web(url_seed):
    data_base = {}
    tocrawl = [url_seed]
    crawled = []
    

    # leitura do arquivo xml
    with open('sitemap.xml') as file_object:
        print('lendo sitemap.xml ...')
        for line in file_object:
            soup = BeautifulSoup(line, "html.parser")
        
            for link in soup.find_all('loc'):
                tocrawl.append(str(link.get_text())) 

    print("leiutra do xml finalizada ... iniciando buscas...\n")
    

    while tocrawl:

        page = tocrawl.pop()

        sys.stdout.write("\rpáginas analisadas: {}/{}.".format(len(crawled), (len(crawled) + len(tocrawl))))
        sys.stdout.flush()
    
        if page not in crawled:
            
            # segue se a página ainda não tiver sido coletada
            
            
            # ignorar páginas que não sejam http
            if page[:5] != "http:":
                crawled.append(page)
                continue
            
            # ignorar páginas de construtora
            if "/construtora" in page:
                crawled.append(page)
                continue
            
            # ignorar páginas de empreendimento
            if '/empreendimento' in page: 
                crawled.append(page)
                continue
            
            # ignorar páginas de bairro
            if '/bairro' in page: 
                crawled.append(page)
                continue
            
            # ignorar páginas com erro de url
            if '.com.brhttps:' in page: 
                crawled.append(page)
                continue
            
            
            # delimitando a busca por (1) apartamento
            # em (2) porto alegre
            # (3) nos três bairros de interesse: auxiliadora, bela vista, mont serrat
            if "-porto-alegre-auxiliadora-apartamento-" not in page:
                if "-porto-alegre-bela-vista-apartamento-" not in page:
                    if "-porto-alegre-mont-serrat-apartamento-" not in page:
                        crawled.append(page)
                        continue               



            
            union(tocrawl, check_links(get_all_links(page)))
            crawled.append(page)
            
            
            #juntando as listas num dataframe
            data_base = pd.DataFrame({'id': id_list,
                                      'price': price_list,
                                      'area': area_list,
                                      'district': district_list,
                                      'city': city_list,
                                      'type': type1_list,
                                      'segment': segment_list,
                                      'condominium': condominium_list,
                                      'iptu': iptu_list,
                                      'rooms': rooms_list,
                                      'box': box_list,
                                      'url': url_list,
                                      'date': date_list
                                     })
            
            data_base.to_csv((date_today + '-foxter.csv'), sep='\t') #salvando csv
            

    print("\nfinalizado todo o site!!")
    print("\ntotal de imóveis salvos: " + str(data_base.shape[0]))

    return
 

In [5]:
# inicialização do web scrapper
date_today = str(date.today())
id_list = []
price_list = []
area_list = []
district_list = []
city_list = []
type1_list = []
segment_list = []
condominium_list = []
iptu_list = []
rooms_list = []
box_list = []
url_list = []
date_list = []



url_seed = ("http://www.foxterciaimobiliaria.com.br") # Página inicial - semente

url_segment = '/imovel/' # identificação de segmento de produto no site



crawl_web(url_seed)

lendo sitemap.xml ...
leiutra do xml finalizada ... iniciando buscas...

páginas analisadas: 27067/27067.
finalizado todo o site!!

total de imóveis salvos: 579
